In [1]:
import sys, os
sys.path.append(os.path.abspath('..'))

from ImageLab import *

import numpy as np
from skimage.feature import hog
from PIL import Image
from PIL import ImageOps

In [2]:
kernel = np.array([[0, 1, 0],
                  [1, 1, 1],
                  [0, 1, 0]])

tkernel1 = np.array([[0, 0, 1, 0, 0],
                    [0, 0, 1, 0, 0],
                    [1, 1, 1, 1, 1],
                    [0, 0, 1, 0, 0],
                    [0, 0, 1, 0, 0]])

xkernel3 = np.array([[1, 1, 0, 0, 0, 1, 1],
                    [1, 1, 1, 0, 1, 1, 1],
                    [0, 1, 1, 1, 1, 1, 0],
                    [0, 0, 1, 1, 1, 0, 0],
                    [0, 1, 1, 1, 1, 1, 0],
                    [1, 1, 1, 0, 1, 1, 1],
                    [1, 1, 0, 0, 0, 1, 1]])


def pipeline(img_path):
    
    gray = Image.open(img_path).convert('L')
    # gray = ImageOps.equalize(gray)
    gray = ImageOps.autocontrast(gray)
    
    gray = np.array(gray)
    inverted = ImageColorSpace(gray, plot=False).process(inversion())
    # stretched = ImageFilters(inverted, plot=False).process(contrast_stretch())
    gamma = ImageFilters(inverted, plot=False).process(gamma_correction(3))
    # equalization = ImageFilters(stretched, plot=True).process(histogram_equalization())
    
    dilated = ImageProcessor(inverted).process(Dilation(), xkernel3)
    closed = ImageProcessor(dilated).process(Erosion(), xkernel3)
    
    sauvola = ImageSegment(inverted, plot=True).process(Pixel_Filter(3, 50, 'Sauvola'))
    
    seggrey = inverted * sauvola

    seggrey = np.squeeze(seggrey)
    return seggrey

def seg_pipeline(img_path):
    gray = Image.open(img_path).convert('L')
    # gray = ImageOps.equalize(gray)
    gray = ImageOps.autocontrast(gray)
    
    gray = np.array(gray)
    inverted = ImageColorSpace(gray, plot=False).process(inversion())
    # stretched = ImageFilters(inverted, plot=False).process(contrast_stretch())
    gamma = ImageFilters(inverted, plot=False).process(gamma_correction(3))
    # equalization = ImageFilters(stretched, plot=True).process(histogram_equalization())
    
    dilated = ImageProcessor(inverted).process(Dilation(), kernel)
    closed = ImageProcessor(dilated).process(Erosion(), kernel)
    
    sauvola = ImageSegment(inverted, plot=True).process(Pixel_Filter(3, 50, 'Sauvola'))
    return sauvola

In [3]:
# segimage = pipeline('test_images/T12.jpg')

In [4]:
# subimage = ImageFilters(segimage, plot=True).process(subimage(300, 360, 250, 300))
# ImageUtil(subimage).save_image_to_folder('reference', '4.png')

In [5]:
import os

# set the path to your test images folder
target_folder = 'test_images'
ref_folder = 'reference'
output_folder = 'detection_images'

chi_difference = 5

# loop through each image file in the folder
for file_name in os.listdir(target_folder):
    # check if the file is an image file
    if file_name.endswith('.jpg') or file_name.endswith('.png'):
        
        img_path = os.path.join(target_folder, file_name)
        file_name = os.path.splitext(file_name)[0]
        
        segimg = pipeline(img_path)
        _=ImageUtil(segimg).save_image_to_folder('Morphological6', f'{file_name}.png')
        
        for test_name in os.listdir(ref_folder):
            if test_name.endswith('.jpg') or test_name.endswith('.png'):
                ref_img = ImageUtil().load_image(ref_folder, test_name)
                test_name = os.path.splitext(test_name)[0]
                detection_image = ImageFilters(segimg).process(HOG_Parallel(ref_img, cell_size=(5,5), block_size=(1, 1)))
                ImageUtil(detection_image).save_image_to_folder(output_folder, f'{file_name}_{test_name}.png')
                
        
    

c:\Programs\Image Processing\Color Space\ImageLab\filters.py:189: RuntimeWarning: invalid value encountered in true_divide
  return 0.5 * np.sum((hist1 - hist2) ** 2 / (hist1 + hist2))
